In [573]:
# !pip install pyecharts
# !pip install selenium
import numpy as np 
import matplotlib.pyplot as plt 
#!pip install yfinance
import yfinance as yf # https://pypi.org/project/yfinance/
import math
from numpy.core.numeric import ones_like
import random
import seaborn as sns
import datetime
import pandas as pd
from pyecharts.charts import Bar
from pyecharts import options as opts
import requests
from bs4 import BeautifulSoup

In [574]:
# get data by ticker-name, start-time & end-time
def get_df_data(ticker_name="AAPL", start_time="2022-01-01", end_time="2022-10-09"):
  df_data = yf.download(tickers=ticker_name, start=start_time, end=end_time) 
  return df_data

def draw_2lines(x, y1, tn1, y2, tn2, ls1=50, le1=160, ls2=50, le2=160):
    data_1 = y1 #np.sin(x)
    data_2 = y2 #np.cos(x) 

    # Create Plot
    fig, ax1 = plt.subplots(figsize=(25, 10)) 
    ax1.set_xlabel('Time range') 
    ax1.set_ylabel(tn1, color = 'red') 
    ax1.plot(x, data_1, color = 'red', label=tn1) 
    ax1.tick_params(axis ='y', labelcolor = 'red') 
    ax1.set_ylim(ls1, le1)
    ax2 = ax1.twinx() 
    ax2.set_ylabel(tn2, color = 'blue') 
    ax2.plot(x, data_2, color = 'blue', label=tn2) 
    ax2.tick_params(axis ='y', labelcolor = 'blue') 
    ax2.set_ylim(ls2, le2)
    
    plt.legend()
    plt.show()
    return 

def time_point_dic(time_list, point_close):
    dic = {}
    i = 0
    while i<len(time_list):
        dic[time_list[i]] = point_close[i]
        i += 1
    return dic

def data_union(dic1, dic2):
    # add in complement time values
    time1 = list(dic1.keys())
    time2 = list(dic2.keys())
    time_union = sorted(list(set(time1+time2)))
    for t in time_union:
        if t not in dic1:
            dic1[t] = -1
        if t not in dic2:
            dic2[t] = -1
    # use previous point to fill default point values
    x = sorted(time_union)
    i = 0
    while i<len(x)-1:
        ct = x[i]
        nt = x[i+1]
        cv1 = dic1[ct]
        nv1 = dic1[nt]
        if nv1<0:
            dic1[nt] = cv1
            nv1 = dic1[nt]
        cv2 = dic2[ct]
        nv2 = dic2[nt]
        if nv2<0:
            dic2[nt] = cv2
            nv2 = dic2[nt]
        i += 1
    return x

In [575]:
st, et = "2022-09-01", "2022-12-31"

tn1 = "BABA" # BABA, BIDU
df_data1 = get_df_data(ticker_name=tn1, start_time=st, end_time=et)
df_data1

tn2 = "9988.HK" # 9988.HK, 9888.HK
df_data2 = get_df_data(ticker_name=tn2, start_time=st, end_time=et)
df_data2

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-09-01,93.849998,95.550003,91.849998,92.199997,92.199997,41471390
2022-09-02,92.000000,92.750000,90.199997,90.650002,90.650002,40477868
2022-09-05,89.500000,89.500000,87.550003,88.500000,88.500000,35505940
2022-09-06,88.849998,89.500000,87.199997,88.050003,88.050003,29242315
2022-09-07,86.099998,87.849998,85.949997,87.400002,87.400002,28479263
...,...,...,...,...,...,...
2022-12-19,86.900002,90.800003,86.400002,87.050003,87.050003,70038366
2022-12-20,85.000000,85.550003,82.500000,84.050003,84.050003,57291122
2022-12-21,84.449997,85.400002,83.699997,84.449997,84.449997,31475853


In [576]:
dic1_open = time_point_dic(list(df_data1.index), list(df_data1['Open']))
dic1_close = time_point_dic(list(df_data1.index), list(df_data1['Close']))
dic1_low = time_point_dic(list(df_data1.index), list(df_data1['Low']))
dic1_high = time_point_dic(list(df_data1.index), list(df_data1['High']))

dic2_open = time_point_dic(list(df_data2.index), list(df_data2['Open']))
dic2_close = time_point_dic(list(df_data2.index), list(df_data2['Close']))
dic2_low = time_point_dic(list(df_data2.index), list(df_data2['Low']))
dic2_high = time_point_dic(list(df_data2.index), list(df_data2['High']))

print( "2 stocks:\t")
print( "\t", len(dic1_open), len(dic1_close), len(dic1_low), len(dic1_high) )
print( "\t", len(dic2_open), len(dic2_close), len(dic2_low), len(dic2_high) )

2 stocks:	
	 79 79 79 79
	 80 80 80 80


In [577]:
x1 = data_union(dic1_open, dic2_open)
x2 = data_union(dic1_close, dic2_close)
x3 = data_union(dic1_low, dic2_low)
x4 = data_union(dic1_high, dic2_high)

print( "\t", len(dic1_open), len(dic1_close), len(dic1_low), len(dic1_high) )
print( "\t", len(dic2_open), len(dic2_close), len(dic2_low), len(dic2_high) )

	 82 82 82 82
	 82 82 82 82


In [578]:
x = x1
y1_open, y1_close, y1_low, y1_high = [], [], [], []
y2_open, y2_close, y2_low, y2_high = [], [], [], []

for t in x:
    y1_open.append( round(dic1_open[t], 2) )
    y1_close.append( round(dic1_close[t], 2) )
    y1_low.append( round(dic1_low[t], 2) )
    y1_high.append( round(dic1_high[t], 2) )
    #
    y2_open.append( round(dic2_open[t], 2) )
    y2_close.append( round(dic2_close[t], 2) )
    y2_low.append( round(dic2_low[t], 2) )
    y2_high.append( round(dic2_high[t], 2) )
    

print( "2 stocks:\t", len(x) )
print( "\t", len(y1_open), len(y1_close), len(y1_low), len(y1_high) )
print( "\t", len(y2_open), len(y2_close), len(y2_low), len(y2_high) )

2 stocks:	 82
	 82 82 82 82
	 82 82 82 82


In [579]:
"""
# data visualization
ls1=ls2=60 
le1=le2=100

y1, y2 = y1_open, y2_open
draw_2lines(x, y1, tn1, y2, tn2, ls1, le1, ls2, le2)
"""

# data visualization: file:///C:/Users/Admin/lines.html
line = (
    Line(init_opts=opts.InitOpts(width="2500px", height="1200px"))
    .add_xaxis(x)
    .add_yaxis(tn1+"_open", y1_open)
    .add_yaxis(tn1+"_close", y1_close)
    .add_yaxis(tn1+"_low", y1_low)
    .add_yaxis(tn1+"_high", y1_high)
    .add_yaxis(tn2+"_open", y2_open)
    .add_yaxis(tn2+"_close", y2_close)
    .add_yaxis(tn2+"_low", y2_low)
    .add_yaxis(tn2+"_high", y2_high)
    .set_global_opts(title_opts=opts.TitleOpts(title="stocks comparison"), 
                     tooltip_opts=opts.TooltipOpts(trigger="axis"),
                    yaxis_opts=opts.AxisOpts(name='宽度',splitline_opts=opts.SplitLineOpts(is_show=True),min_=60)) # , subtitle="商店A中六樣商品數"
)
line.render("lines.html")

'C:\\Users\\Admin\\lines.html'

In [580]:
# 
import datetime
def get_dates_from_range(start_date, end_dates):
    start = datetime.datetime.strptime(start_date, "%d-%m-%Y") 
    end = datetime.datetime.strptime(end_dates, "%d-%m-%Y")
    date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
    date_list = []
    for date in date_generated:
        date_ymd = date
        y, m, d = date_ymd.year, date_ymd.month, date_ymd.day
        ymd = str(y) + "-"
        if m<10:
            ymd = ymd + "0" + str(m)
        else:
            ymd = ymd + str(m)
        ymd = ymd + "-"
        if d<10:
            ymd = ymd + "0" + str(d)
        else:
            ymd = ymd + str(d)
        date_list.append( ymd  )
    return date_list



In [581]:
ticker_name = '9988.HK'
start_time = '2022-12-19'
end_time = '2022-12-20'

date_list = get_dates_from_range('04-12-2022', '24-12-2022')
date_list

df_list = []

nn = 0
while nn<len(date_list)-1: # 
    start_time, end_time = date_list[nn], date_list[nn+1]
    #
    interval_len = '1m'
    df_data = yf.download(tickers=ticker_name, start=start_time, end=end_time, interval=interval_len)
    if len(df_data)>0:
        max( list(df_data['Close']) ), max( list(df_data['Open']) ), max( list(df_data['Low']) ), max( list(df_data['High']) )

        info_list = []
        i = 0
        open_point = df_data.iloc[0]['Open']
        while i<len(df_data):
            current_time = df_data.index[i]
            current_it = df_data.iloc[i]
            current_point = current_it['Close']
                # initialization
            future_max_time = current_time
            future_max_point = current_point 
            j = i + 1
            while j<len(df_data):
                future_time = df_data.index[j]
                future_it = df_data.iloc[j]
                future_point = future_it['Close']
                if future_max_point<future_point:
                    future_max_point = future_point
                    future_max_time = future_time
                j += 1
            ratio = (future_max_point - current_point)/current_point * 100
            #print( current_time, round(current_point, 2), future_max_time, round(future_max_point, 2), '\t', round(ratio, 2) )
            info_list.append( [current_time, open_point, current_point, future_max_time, future_max_point, ratio] )
            i += 1
        new_df = pd.DataFrame(info_list, columns = ['current_time', 'open_point', 'current_point', 'future_max_time', 'future_max_point', 'ratio'])

        max_info = new_df.sort_values(by=['ratio'], ascending=False)
        print(start_time, end_time, "\t")
        #print(max_info.head(3))
        #print()
        df_list.append( max_info.head(3) )
    nn += 1

useful_info = pd.concat( df_list )
useful_info
#useful_info.to_csv('C:/Users/Admin/Desktop/stocks_analyze_predict/daily_profit.csv')

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 9988.HK: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
2022-12-05 2022-12-06 	
[*********************100%***********************]  1 of 1 completed
2022-12-06 2022-12-07 	
[*********************100%***********************]  1 of 1 completed
2022-12-07 2022-12-08 	
[*********************100%***********************]  1 of 1 completed
2022-12-08 2022-12-09 	
[*********************100%***********************]  1 of 1 completed
2022-12-09 2022-12-10 	
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 9988.HK: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- 9988.HK: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 co

,current_time,open_point,current_point,future_max_time,future_max_point,ratio
3,2022-12-05 09:33:00,88.699997,87.400002,2022-12-05 15:56:00,92.800003,6.178491
1,2022-12-05 09:31:00,88.699997,87.699997,2022-12-05 15:56:00,92.800003,5.815287
5,2022-12-05 09:35:00,88.699997,87.750000,2022-12-05 15:56:00,92.800003,5.754989
0,2022-12-06 09:30:00,88.199997,88.949997,2022-12-06 10:19:00,91.699997,3.091625
1,2022-12-06 09:31:00,88.199997,89.050003,2022-12-06 10:19:00,91.699997,2.975849
3,2022-12-06 09:33:00,88.199997,89.250000,2022-12-06 10:19:00,91.699997,2.745095
7,2022-12-07 09:37:00,89.099998,88.750000,2022-12-07 13:19:00,91.849998,3.492956
6,2022-12-07 09:36:00,89.099998,88.800003,2022-12-07 13:19:00,91.849998,3.434679
4,2022-12-07 09:34:00,89.099998,88.900002,2022-12-07 13:19:00,91.849998,3.318332
2,2022-12-08 09:32:00,86.800003,86.449997,2022-12-08 11:21:00,90.849998,5.089649


In [582]:
###
#!pip3 install pyautogui
#!pip install keyboard
import pyautogui
import keyboard
from tkinter import Tk
import datetime
#print(pyautogui.size()) # screen: Size(width=1920, height=1080)

def draw_lines_html(x, y1, y2, title_name="stocks comparison"):
    line = (
        Line(init_opts=opts.InitOpts(width="1500px", height="800px"))
        .add_xaxis(x)
        .add_yaxis("BABA-Close", y1)
        .add_yaxis("9988_real-quote", y2)
        .set_global_opts(title_opts=opts.TitleOpts(title=title_name), 
                         tooltip_opts=opts.TooltipOpts(trigger="axis"),
                        yaxis_opts=opts.AxisOpts(name='width',splitline_opts=opts.SplitLineOpts(is_show=True),min_=80)) # , subtitle="商店A中六樣商品數"
    )
    line.render("lines01.html")
    return 

def swtich_right_desktop():
    keyboard.press("ctrl")
    keyboard.press("windows")
    keyboard.press("right")
    keyboard.release("ctrl")
    keyboard.release("windows")
    keyboard.release("right")
    return 

def swtich_left_desktop():
    keyboard.press("ctrl")
    keyboard.press("windows")
    keyboard.press("left")
    keyboard.release("ctrl")
    keyboard.release("windows")
    keyboard.release("left")
    return 

def move_mouse(posx, posy, time_duration=0.2):
    pyautogui.moveTo(posx, posy, duration=time_duration)
    return 

def mouse_left_click(posx, posy):
    pyautogui.click(posx, posy)
    return 

def keyboard_copy():
    keyboard.press("ctrl")
    keyboard.press("c")
    keyboard.release("ctrl")
    keyboard.release("c")
    return 

def switch_window(posx=600, posy=300, time_duration=0.2):
    keyboard.press("alt")
    keyboard.press("tab")
    keyboard.release("alt")
    keyboard.release("tab")
    pyautogui.moveTo(posx, posy, duration=time_duration)
    pyautogui.click(posx, posy)
    return 

def recording(every_line):
    f = open("C:/Users/Admin/Desktop/stocks_analyze_predict/database_quote01.txt", "a")
    f.write(every_line + "\n")
    f.close()
    return 

In [585]:
x, y1, y2 = [], [], []
refer_close = 87.97

for i in range(20):
        #
    swtich_right_desktop()
        # move mouse
    posx, posy = 1190, 1045
    pyautogui.moveTo(posx, posy, duration = 0.2)
    pyautogui.click(posx, posy)
        #
    posx, posy = 1100, 510
    pyautogui.click(posx, posy)
    pyautogui.typewrite(["9", "9", "8", "8", "enter"])
        #
    posx, posy = 1100, 568
    pyautogui.moveTo(posx, posy, duration = 0.3)
    pyautogui.click(posx, posy)
    pyautogui.click(posx, posy)
    keyboard_copy() # https://www.bochk.com/tc/investment/econanalysis/infostock.html
    #
    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    current_quote = float(Tk().clipboard_get()) # get content from the clip board
    difference_ratio = (refer_close-current_quote)/current_quote*100    
    #
    print( i, "\t", current_time, "\t", refer_close, current_quote, round(difference_ratio, 2) )
    #
    switch_window()
    swtich_left_desktop()
    #
    # data visualization: file:///C:/Users/Admin/lines.html
    x.append( current_time )
    y1.append( refer_close )
    y2.append( current_quote )
    limit_len = 30
    if len(x)>limit_len:
        draw_lines_html(x[-limit_len:], y1[-limit_len:], y2[-limit_len:], str(i+1))
    else:
        draw_lines_html(x, y1, y2, str(i+1))
    # refresh webpage
    posx, posy = 80, 50
    pyautogui.moveTo(posx, posy, duration = 0.1)
    pyautogui.click(posx, posy)
    #write to file
    every_line = str(refer_close) + "\t" + str(current_time) + "\t" + str(current_quote) + "\t" + str(round(difference_ratio, 2))
    recording(every_line)
    #
    time.sleep(7)
#
print("monitor finished")

0 	 2022-12-23 15:57:43 	 87.97 86.85 1.29
1 	 2022-12-23 15:57:52 	 87.97 86.85 1.29
2 	 2022-12-23 15:58:02 	 87.97 86.75 1.41
3 	 2022-12-23 15:58:11 	 87.97 86.8 1.35
4 	 2022-12-23 15:58:20 	 87.97 86.8 1.35
5 	 2022-12-23 15:58:29 	 87.97 86.8 1.35
6 	 2022-12-23 15:58:39 	 87.97 86.75 1.41
7 	 2022-12-23 15:58:48 	 87.97 86.8 1.35
8 	 2022-12-23 15:58:57 	 87.97 86.75 1.41
9 	 2022-12-23 15:59:07 	 87.97 86.75 1.41
10 	 2022-12-23 15:59:16 	 87.97 86.75 1.41
11 	 2022-12-23 15:59:25 	 87.97 86.8 1.35
12 	 2022-12-23 15:59:35 	 87.97 86.75 1.41
13 	 2022-12-23 15:59:44 	 87.97 86.75 1.41
14 	 2022-12-23 15:59:53 	 87.97 86.75 1.41
15 	 2022-12-23 16:00:03 	 87.97 86.75 1.41
16 	 2022-12-23 16:00:12 	 87.97 86.75 1.41
17 	 2022-12-23 16:00:22 	 87.97 86.75 1.41
18 	 2022-12-23 16:00:31 	 87.97 86.75 1.41
19 	 2022-12-23 16:00:40 	 87.97 86.75 1.41
monitor finished
